#### Part 1: Install and Import Dependencies

In [ ]:
# Install boto3 if not already installed
%pip install boto3 botocore

In [ ]:
# Import the necessary libraries
import boto3
import botocore.config
import json
from datetime import datetime

#### Part 2: Function to Generate Quotes

In [ ]:
def quote_generate_using_bedrock(topic: str) -> str:
    """Generates a quote using Bedrock."""
    
    prompt = f"""<s>[INST]Human: Provide a motivational quote on the topic "{topic}".
    Assistant:[/INST]
    """
    
    body = {
        "prompt": prompt,
        "max_gen_len": 128,  # Reduced length since quotes are shorter
        "temperature": 0.7,  # Slightly increased for more creative outputs
        "top_p": 0.9
    }

    try:
        bedrock = boto3.client(
            "bedrock-runtime", 
            region_name="us-east-1",
            config=botocore.config.Config(
                read_timeout=300, 
                retries={'max_attempts': 3}
            )
        )
        response = bedrock.invoke_model(
            body=json.dumps(body), 
            modelId="meta.llama2-13b-chat-v1"
        )

        response_content = response.get('body').read()
        response_data = json.loads(response_content)
        quote = response_data.get('generation', '')
        
        if quote:
            return quote
        else:
            raise ValueError("No quote content in the response.")
    
    except Exception as e:
        print(f"Error generating the quote: {e}")
        return ""

In [1]:
# Test the quote generation function
'''
generated_quote = quote_generate_using_bedrock(topic="success")
print(generated_quote)
'''

'\ngenerated_quote = quote_generate_using_bedrock(topic="success")\nprint(generated_quote)\n'

#### Part 3: Function to Save Quote to S3

In [ ]:
def save_quote_to_s3(s3_key: str, s3_bucket: str, quote: str) -> None:
    """Saves the generated quote to an S3 bucket."""
    
    s3 = boto3.client('s3')

    try:
        s3.put_object(Bucket=s3_bucket, Key=s3_key, Body=quote)
        print("Quote saved to S3 successfully.")
    
    except Exception as e:
        print(f"Error saving the quote to S3: {e}")

In [ ]:
##### Test saving to S3 by running:
'''
#Save the generated quote to S3
if generated_quote:
    current_time = datetime.now().strftime('%H%M%S')
    s3_key = f"quote-output/{current_time}.txt"
    s3_bucket = 'aws_bedrock_course1'  # Replace with your actual bucket
    save_quote_to_s3(s3_key, s3_bucket, generated_quote)
'''

#### Part 4: Lambda Handler Function for Integration

In [ ]:
def lambda_handler(event, context):
    """Simulates AWS Lambda handler to generate a quote and save it to S3."""
    
    try:
        event_data = json.loads(event['body'])
        topic = event_data['topic']  # Expecting "topic" in the input
        
        generated_quote = quote_generate_using_bedrock(topic=topic)

        if generated_quote:
            current_time = datetime.now().strftime('%H%M%S')
            s3_key = f"quote-output/{current_time}.txt"
            s3_bucket = 'aws_bedrock_course1'  # Replace with your actual bucket
            
            save_quote_to_s3(s3_key, s3_bucket, generated_quote)
        
        else:
            print("No quote was generated.")
        
        return {
            'statusCode': 200,
            'body': json.dumps('Quote generation completed successfully.')
        }

    except KeyError as e:
        print(f"Key error: {e}")
        return {
            'statusCode': 400,
            'body': json.dumps(f"Invalid input data: {e}")
        }
    
    except Exception as e:
        print(f"Unexpected error: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps('An error occurred during quote generation.')
        }

In [ ]:
# Simulate an event for lambda handler
'''
mock_event = {
    'body': json.dumps({'topic': 'perseverance'})
}

# Test the lambda handler function
lambda_handler(mock_event, None)
'''